In [1]:
import sys


no_of_cpu =8
max_cores =8
executor_mem = '56g'


Job_Name = 'Ajeet  :Account_agg'


from pyspark.mllib.stat import Statistics
from pyspark.sql.functions import asc,lit
#warnings.filterwarnings('error')
import pyspark
from datetime import datetime,timedelta
from pyspark.sql import SQLContext
from pyspark import SparkContext, SparkConf
from pyspark.sql import functions as F

conf = pyspark.SparkConf()
#import numpy
import calendar
#import pandas as pd
#import simplejson as json
#import pandas as pd
import numpy as np
import datetime
from pyspark.sql.functions import *
from pyspark.sql.types import DateType
from pyspark.sql.functions import lit
import simplejson as json
import json, pprint, requests
#es_nodes = '10.35.12.5'
#es_nodes = '10.35.12.6'
es_nodes = '10.35.12.194'#,10.35.12.6,10.35.12.5
es_nodes_temp='10.35.12.194'
es_port = '5432'
es_user = 'elastic'
es_pwd = 'bEiilauM3es'
mesos_ip = 'mesos://10.33.195.18:5050'#'mesos://10.35.12.5:5050'

spark.stop() #############NEED TO COMMENT THIS SPARK.STOP WHEN RUNNING THROUGH SHELL###############################

conf.setMaster(mesos_ip)

conf.set('spark.executor.cores',no_of_cpu) ### change 1
#conf.set('spark.memory.fraction','.2')
conf.set('spark.executor.memory',executor_mem) 
conf.set('spark.es.scroll.size','10000')
conf.set('spark.network.timeout','600s')
conf.set('spark.sql.crossJoin.enabled', 'true')

conf.set('spark.ui.port','4052')

conf.set('spark.executor.heartbeatInterval','60s')
conf.set("spark.driver.cores","4")
conf.set("spark.driver.extraJavaOptions","-Xmx4g -Xms4g")

#conf.set("spark.shuffle.blockTransferService", "nio"); 
conf.set("spark.files.overwrite","true");
conf.set("spark.kryoserializer.buffer", "70"); 
conf.set("spark.driver.extraJavaOptions", "-XX:+UseG1GC");
conf.set("spark.executor.extraJavaOptions", "-XX:+UseG1GC");
conf.set("spark.serializer", "org.apache.spark.serializer.KryoSerializer"); 
conf.set("spark.broadcast.compress", "true"); 
conf.set("spark.shuffle.compress", "true"); 
conf.set("spark.shuffle.spill.compress", "true");
conf.set("spark.app.name", Job_Name)
#conf.set("spark.io.compression.codec","org.apache.spark.io.LZ4CompressionCodec");
#conf.set("spark.sql.inMemoryColumnarStorage.compressed", "true"); 
from pyspark.sql.functions import broadcast

conf.set('spark.driver.memory','20g') ### change 2
conf.set('spark.cores.max',max_cores) ### change 3
conf.set('spark.sql.shuffle.partitions','400')
#conf.set('spark.sql.crossJoin.enabled', 'true')
conf.set('es.nodes',es_nodes)
conf.set('es.port',es_port)
conf.set('es.nodes.wan.only','true')
conf.set("spark.sql.autoBroadcastJoinThreshold",-1)

#conf.set('spark.es.net.http.auth.user','Spark')
#conf.set('spark.es.net.http.auth.pass','Jarkpet1Sap3')
conf.set('spark.num.executors','4')
conf.set('spark.debug.maxToStringFields', 200)

conf.set('spark.es.net.http.auth.user', es_user)
conf.set('spark.es.net.http.auth.pass', es_pwd)

conf.set('spark.es.mapping.date.rich','false')
spark = pyspark.SparkContext(conf=conf)
sqlContext = SQLContext(spark)

# Load Data into PySpark DataFrames
# Prodcom Data Frame
import json, pprint, requests
import pyspark.sql.functions as sf
from pyspark.sql.types import *


import numpy as np
import datetime
from pyspark.sql.functions import *
from pyspark.sql.types import DateType



Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
15069,None,pyspark,idle,,,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [2]:
prod_url = "jdbc:postgresql://10.35.12.194:5432/gpadmin"
prod_host = '10.35.12.194'
prod_port = '5432'
prod_dbname = 'gpadmin'
user_prod="gpspark"
pwd_prod="spark@456"
dbschema="public"


prod_gpdb_spark_options ={
    "url": "jdbc:postgresql://{host}:{port}/{dbname}".format(host=prod_host,port=prod_port, dbname=prod_dbname),
    "user": "{user}".format(user=user_prod),
    "password": "{password}".format(password=pwd_prod)
} 

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [3]:
prod_url = "jdbc:postgresql://10.35.12.194:5432/gpadmin"
prod_host = '10.35.12.194'
prod_port = '5432'
prod_dbname = 'gpadmin'
user_prod="gpcustcoe"
pwd_prod="gpcustcoe@123"
dbschema="public"


prod_gpdb_spark_options ={
    "url": "jdbc:postgresql://{host}:{port}/{dbname}".format(host=prod_host,port=prod_port, dbname=prod_dbname),
    "user": "{user}".format(user=user_prod),
    "password": "{password}".format(password=pwd_prod)
} 

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [4]:
# loading table as dataframe
def load_gpdb_jdbc(col_str,dbtable,dbschema,col_name=None,time_filter=None,partitionColumn="row_num"):
    """
    This is used to read gpdb with filter for columns and can apply other filter(date,values).
    Time filter contains startdate,enddate
    """
    gscPythonOptions = {
                        "url": prod_url,
                        "user": user_prod,
                        "password": pwd_prod,
                        "dbschema": dbschema,
                        "dbtable": dbtable,
                        "partitionColumn":partitionColumn,
                        "partitions": 8,
                        "server.port":"1150-1170"}
   
    if time_filter:
        data = sqlContext.read.format("greenplum").options(**gscPythonOptions).load()\
                .selectExpr(col_str).drop_duplicates().filter(col(col_name).between(to_timestamp(lit(time_filter['start_date']),
                                                                 format='yyyy-MM-dd'),
                                                    to_timestamp(lit(time_filter['end_date']),
                                                                 format='yyyy-MM-dd')))
    else :
        data = sqlContext.read.format("greenplum").options(**gscPythonOptions).load()\
                .selectExpr(col_str).drop_duplicates()
   
    return data

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [36]:
col_str=['policy_no', 'product_name','product_cd','pol_incept_date', 'pol_exp_date','ncb_percent', 'sum_insured', 'lob as insurance_type', 'premiumamount',
        'hypothecator','registration_no','veh_make','veh_model','pol_status_cd','ins_name_client_no', 'binddate', 'certificate_no', 'mfg_year']

sad=load_gpdb_jdbc(col_str,"sales_all_details","datamarts")\
.filter(col('binddate')>='2023-01-01')\
.filter(col('product_name').isin('PREMIER - MEDICARE','MOTORPRIVATECARPOLICYINSURANCE','MEDICAREPLUS',
                                 'PRIVATECARINSURANCEPOLICY','MEDICARE','MOTORTWOWHEELERPOLICY','PROTECT - MEDICARE'))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [37]:
col_str=['source_customer_id','dc_unified_id']

mbdp=load_gpdb_jdbc(col_str,"mastercraft_bulk_detail_prod", "public")
mbdp=mbdp.filter((col('dc_unified_id').isNotNull()))\
                        .filter((col('source_customer_id').isNotNull()))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [38]:
df=sad.join(mbdp, sad.ins_name_client_no==mbdp.source_customer_id, "inner")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [39]:
col_str=['dc_unified_id','dc_active_flag','full_name as name','mobile_primary','mobile_secondary', 'psnt_address_line_1', 'psnt_address_line_2', 'psnt_address_line_3']
mmp=load_gpdb_jdbc(col_str,"mastercraft_master_prod", "public").filter((col('dc_active_flag').isin('1')))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [40]:
df=df.join(mmp, df.dc_unified_id==mmp.dc_unified_id, 'inner').drop(mmp.dc_unified_id)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [41]:
col_str=['txt_uin_number as uin_number', 'txt_product_code' ]
aud=load_gpdb_jdbc(col_str,"account_gc_accmst_uin_details", "public")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [42]:
df=df.join(aud, df.product_cd==aud.txt_product_code, 'left')

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [43]:
col_str=['txt_payer_customer_id','num_acceptance_voucher_no as trans_id','dat_acceptance_date as trans_date','txt_remarks as narration','num_instrument_amt as trans_amt']
acp=load_gpdb_jdbc(col_str,"account_gc_acc_payment_entry", "public")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [44]:
df=df.join(acp, df.ins_name_client_no==acp.txt_payer_customer_id, "left")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [45]:
df=df.withColumn('mobile', coalesce('mobile_primary', 'mobile_secondary'))\
.withColumn('coverName', col('product_name'))\
.withColumn('coverstartdate', col('pol_incept_date'))\
.withColumn('sum_assured', col('sum_insured'))\
.withColumn('financier_name', col('hypothecator'))\
.withColumn('tenure_months', round(months_between(col('pol_exp_date'),col('pol_incept_date')),0))\
.withColumn('tenure_yrs', round(months_between(col('pol_exp_date'),col('pol_incept_date'))/lit(12),0))\
.withColumn('hypothecation', when(col('hypothecator').isNotNull(), lit("Y")).otherwise(lit("N")))\
.withColumn('coverenddate', col('pol_exp_date'))\
.withColumn('address', concat_ws('',trim(col('psnt_address_line_1')),trim(col('psnt_address_line_2')), trim(col('psnt_address_line_3'))))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [46]:
df=df.withColumn('maskedPolicyNumber',lit(None).cast('string'))\
.withColumn('dob',lit(None).cast('string'))\
.withColumn('landline',lit(None).cast('string'))\
.withColumn('email',lit(None).cast('string'))\
.withColumn('pan',lit(None).cast('string'))\
.withColumn('kycCompliance',lit(None).cast('string'))\
.withColumn('nominee',lit(None).cast('string'))\
.withColumn('coverDescription',lit(None).cast('string'))\
.withColumn('policyName',lit(None).cast('string'))\
.withColumn('policyDescription',lit(None).cast('string'))\
.withColumn('tenureDays',lit(None).cast('string'))\
.withColumn('policyType',lit(None).cast('string'))\
.withColumn('premiumFrequency',lit(None).cast('string'))\
.withColumn('premiumPaymentYears',lit(None).cast('string'))\
.withColumn('premiumPaymentMonths',lit(None).cast('string'))\
.withColumn('nextPremiumDueDate',lit(None).cast('string'))\
.withColumn('policyStatus',lit(None).cast('string'))\
.withColumn('rtoLocation',lit(None).cast('string'))\
.withColumn('startDate',lit(None).cast('string'))\
.withColumn('endDate',lit(None).cast('string'))\
.withColumn('type',lit(None).cast('string'))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [47]:
df=df.select('maskedPolicyNumber','name','dob','mobile','landline','address','email', 'pan', 'kycCompliance', 'nominee',
                                                      'uin_number', 'covername','sum_assured', 'sum_insured','tenure_yrs', 'tenure_months', 'premiumamount',
                                                      'coverstartdate', 'coverenddate','policyName','policyDescription','tenureDays','pol_incept_date', 'pol_exp_date', 
                                                      'policyType','insurance_type', 'premiumFrequency', 'premiumPaymentYears', 'premiumPaymentMonths', 'nextPremiumDueDate', 'policyStatus',
                                                      'registration_no', 'veh_make', 'veh_model','mfg_year','rtoLocation','ncb_percent','hypothecation', 'financier_name',
                                                      'startDate', 'endDate','trans_id', 'trans_date', 'narration','type','trans_amt', 'pol_status_cd')

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [48]:
df_distinct=df.withColumn('primary_key', concat_ws('',trim(col('dc_unified_id')),trim(col('source_customer_id')), trim(col('policy_no')),
                                                       trim(col('certificate_no')))).dropDuplicates()

from pyspark.sql.window import Window

df_distinct = df_distinct.withColumn("rn", F.row_number()
        .over(Window.partitionBy("primary_key")
        .orderBy(F.col("binddate").desc())))

df_distinct = df_distinct.filter(F.col("rn") == 1).drop("rn").drop("primary_key")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

An error was encountered:
"cannot resolve '`dc_unified_id`' given input columns: [hypothecation, endDate, veh_make, sum_insured, veh_model, mfg_year, landline, policyName, tenureDays, type, pol_status_cd, policyDescription, rtoLocation, dob, nominee, trans_amt, kycCompliance, coverenddate, covername, registration_no, premiumamount, nextPremiumDueDate, coverstartdate, startDate, tenure_yrs, pol_exp_date, policyStatus, tenure_months, pan, premiumFrequency, trans_date, email, address, policyType, financier_name, premiumPaymentMonths, mobile, pol_incept_date, insurance_type, name, maskedPolicyNumber, premiumPaymentYears, uin_number, ncb_percent, trans_id, sum_assured, narration];;\n'Project [maskedPolicyNumber#7761, name#6867, dob#7807, mobile#7356, landline#7854, address#7716, email#7902, pan#7951, kycCompliance#8001, nominee#8052, uin_number#7017, covername#7392, sum_assured#7467, sum_insured#4379, tenure_yrs#7587, tenure_months#7546, premiumamount#4420, coverstartdate#7429, coverenddate

In [49]:
df_distinct.persist()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

An error was encountered:
name 'df_distinct' is not defined
Traceback (most recent call last):
NameError: name 'df_distinct' is not defined



In [ ]:
df_distinct.filter(col('policy_no').isin(''))

In [ ]:
#df_distinct.count()

In [ ]:
output_index = 'account_aggregator_data_staging'
schema = 'customermart'

import sys
try:
    
     df_distinct.write.option("truncate", "true").format("greenplum")\
    .option("dbtable",output_index).option('dbschema',schema)\
    .option("server.port","1150-1170").option("url",prod_url)\
    .option("user", user_prod).option("password",pwd_prod).mode('overwrite').save()
    
    
except Exception as e:
    x = e
    print(x)
else:
    x = 'success'
    print(x)

In [51]:
gscPythonOptions = {
         "url": prod_url,
         "user": user_prod,
         "password": pwd_prod,
         "dbschema":'customermart',
         "dbtable": "account_aggregator_data",
         "server.port":"1150-1170"}


account_aggregator_data = sqlContext.read.format("greenplum").options(**gscPythonOptions).load()
                            



FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [52]:
account_aggregator_data=account_aggregator_data.withColumn('maskedPolicyNumber',lit(None).cast('string'))\
.withColumn('dob',lit(None).cast('string'))\
.withColumn('landline',lit(None).cast('string'))\
.withColumn('email',lit(None).cast('string'))\
.withColumn('pan',lit(None).cast('string'))\
.withColumn('kycCompliance',lit(None).cast('string'))\
.withColumn('nominee',lit(None).cast('string'))\
.withColumn('coverDescription',lit(None).cast('string'))\
.withColumn('policyName',lit(None).cast('string'))\
.withColumn('policyDescription',lit(None).cast('string'))\
.withColumn('tenureDays',lit(None).cast('string'))\
.withColumn('policyType',lit(None).cast('string'))\
.withColumn('premiumFrequency',lit(None).cast('string'))\
.withColumn('premiumPaymentYears',lit(None).cast('string'))\
.withColumn('premiumPaymentMonths',lit(None).cast('string'))\
.withColumn('nextPremiumDueDate',lit(None).cast('string'))\
.withColumn('policyStatus',lit(None).cast('string'))\
.withColumn('rtoLocation',lit(None).cast('string'))\
.withColumn('startDate',lit(None).cast('string'))\
.withColumn('endDate',lit(None).cast('string'))\
.withColumn('type',lit(None).cast('string'))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [53]:
account_aggregator_data.columns

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

['policy_no', 'name', 'mobile', 'address', 'uin_number', 'covername', 'tenure_yrs', 'tenure_months', 'coverstartdate', 'coverenddate', 'ncb_percent', 'product_name', 'pol_incept_date', 'pol_exp_date', 'sum_assured', 'sum_insured', 'insurance_type', 'premiumamount', 'hypothecation', 'financier_name', 'registration_no', 'veh_make', 'veh_model', 'pol_status_cd', 'mfg_year', 'trans_id', 'trans_date', 'narration', 'trans_amt', 'maskedPolicyNumber', 'dob', 'landline', 'email', 'pan', 'kycCompliance', 'nominee', 'coverDescription', 'policyName', 'policyDescription', 'tenureDays', 'policyType', 'premiumFrequency', 'premiumPaymentYears', 'premiumPaymentMonths', 'nextPremiumDueDate', 'policyStatus', 'rtoLocation', 'startDate', 'endDate', 'type']

In [54]:
account_aggregator_data=account_aggregator_data.select('maskedPolicyNumber','name','dob','mobile','landline','address','email', 'pan', 'kycCompliance', 'nominee',
                                                      'uin_number', 'covername','coverDescription','sum_assured', 'sum_insured','tenure_yrs', 'tenure_months', 'premiumamount',
                                                      'coverstartdate', 'coverenddate','policyName','policy_no','policyDescription','tenureDays','pol_incept_date', 'pol_exp_date', 
                                                      'policyType','insurance_type', 'premiumFrequency', 'premiumPaymentYears', 'premiumPaymentMonths', 'nextPremiumDueDate', 'policyStatus',
                                                      'registration_no', 'veh_make', 'veh_model','mfg_year','rtoLocation','ncb_percent','hypothecation', 'financier_name',
                                                      'startDate', 'endDate','trans_id', 'trans_date', 'narration','type','trans_amt', 'pol_status_cd')

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [21]:
len(account_aggregator_data.columns)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

48

In [56]:
account_aggregator_data.persist()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

DataFrame[maskedPolicyNumber: string, name: string, dob: string, mobile: string, landline: string, address: string, email: string, pan: string, kycCompliance: string, nominee: string, uin_number: string, covername: string, sum_assured: double, sum_insured: double, tenure_yrs: decimal(38,18), tenure_months: decimal(38,18), premiumamount: double, coverstartdate: date, coverenddate: date, policyName: string, policy_no: string, policyDescription: string, tenureDays: string, pol_incept_date: date, pol_exp_date: date, policyType: string, insurance_type: string, premiumFrequency: string, premiumPaymentYears: string, premiumPaymentMonths: string, nextPremiumDueDate: string, policyStatus: string, registration_no: string, veh_make: string, veh_model: string, mfg_year: string, rtoLocation: string, ncb_percent: double, hypothecation: string, financier_name: string, startDate: string, endDate: string, trans_id: decimal(20,0), trans_date: timestamp, narration: string, type: string, trans_amt: decima

In [ ]:
#account_aggregator_data.count()

In [ ]:
#account_aggregator.filter(col('tenure_yrs').isNotNull()).show()

In [57]:
account_aggregator_data=account_aggregator_data.withColumn('tenure_yrs', col('tenure_yrs').cast('float'))\
.withColumn('tenure_months', col('tenure_months').cast('float'))\
.withColumn('trans_id', col('trans_id').cast('float'))\
.withColumn('trans_amt', col('trans_amt').cast('float'))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [58]:
account_aggregator_data=account_aggregator_data.withColumn('coverstartdate',date_format(to_utc_timestamp(col('coverstartdate'),"UTC"),'yyyy-MM-dd'))\
.withColumn('coverenddate',date_format(to_utc_timestamp(col('coverenddate'),"UTC"),'yyyy-MM-dd'))\
.withColumn('pol_incept_date',date_format(to_utc_timestamp(col('pol_incept_date'),"UTC"),'yyyy-MM-dd'))\
.withColumn('pol_exp_date',date_format(to_utc_timestamp(col('pol_exp_date'),"UTC"),'yyyy-MM-dd'))\
.withColumn('trans_date',from_unixtime(unix_timestamp(col('trans_date')),'yyyy-MM-dd HH:mm:ss'))\
.withColumn('tenure_yrs', round(col('tenure_yrs'), 2))\
.withColumn('tenure_months', round(col('tenure_months'), 2))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [59]:
target_index='account_aggregator_data'
print(target_index)

import json, pprint, requests
es_nodes = '10.35.12.212'
es_port = '9200'
es_user = 'elastic'
es_pwd = 'bEiilauM3es'

######target_index = 'actuarial_premium_risk_tables'----previous name...delete later

#alias_name = 'nepuepr_uat_3'
target_doc_type = 'account_aggregator_data'
#########delete index if already exists########################################################################################################################################################################

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

account_aggregator_data

In [50]:
# #########delete index ###############################################
# headers = {'Content-Type': 'application/json'}
# query = {"query":{"match_all":{}}}
# URL = 'http://'+es_user+':'+es_pwd+'@'+es_nodes+':'+es_port+'/'+target_index
# r = requests.delete(URL, data=json.dumps(query), headers=headers) 
# print(r)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

<Response [200]>

In [60]:
#########create index ##################################################################################################################################################################
headers = {'Content-Type': 'application/json'}
query_create= {"settings": { "index.number_of_shards": "1" }}
URL = 'http://'+es_user+':'+es_pwd+'@'+es_nodes+':'+es_port+'/'+target_index
r = requests.put(URL, data=json.dumps(query_create), headers=headers)
print(r)



FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

<Response [200]>

In [61]:
target_index_doc_type = target_index +'/'+ target_doc_type

target_index_doc_type


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

'account_aggregator_data/account_aggregator_data'

In [62]:
target_index_doc_type

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

'account_aggregator_data/account_aggregator_data'

In [63]:
df_main=account_aggregator_data
df_main=df_main.drop_duplicates()
df_main.columns

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

['maskedPolicyNumber', 'name', 'dob', 'mobile', 'landline', 'address', 'email', 'pan', 'kycCompliance', 'nominee', 'uin_number', 'covername', 'sum_assured', 'sum_insured', 'tenure_yrs', 'tenure_months', 'premiumamount', 'coverstartdate', 'coverenddate', 'policyName', 'policy_no', 'policyDescription', 'tenureDays', 'pol_incept_date', 'pol_exp_date', 'policyType', 'insurance_type', 'premiumFrequency', 'premiumPaymentYears', 'premiumPaymentMonths', 'nextPremiumDueDate', 'policyStatus', 'registration_no', 'veh_make', 'veh_model', 'mfg_year', 'rtoLocation', 'ncb_percent', 'hypothecation', 'financier_name', 'startDate', 'endDate', 'trans_id', 'trans_date', 'narration', 'type', 'trans_amt', 'pol_status_cd']

In [64]:
try:
     df_main.write.format('org.elasticsearch.spark.sql').mode('overwrite').option('es.index.auto.create', 'true').option('es.nodes' , es_nodes).option('es.port', es_port).option('es.resource', target_index_doc_type).save() 
except Exception as e :
    x = e
else: 
    x = 200 # success
print(x)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

200

In [22]:
spark.stop()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…